In [11]:
import praw
import pandas as pd
from datetime import datetime
import time
import requests

In [12]:
# ====================================
# USAR PUSHSHIFT PARA DATOS HISTÓRICOS
# ====================================

def get_pushshift_data(subreddit, keyword, after, before, size=100):
    """
    Obtener datos históricos de Reddit usando Pushshift
    """
    base_url = "https://api.pullpush.io/reddit/search/submission"
    
    params = {
        'subreddit': subreddit,
        'q': keyword,
        'after': int(after.timestamp()),
        'before': int(before.timestamp()),
        'size': size,
        'sort': 'created_utc',
        'sort_type': 'created_utc'
    }
    
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            return response.json()['data']
        else:
            print(f"Error {response.status_code}")
            return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [13]:
# Conexión a Reddit
reddit = praw.Reddit(
    client_id='imEh5Q5S8czyiqOuFoIMig',
    client_secret='UjTZpZLBJy3NxfkFJu2QtQXmhtcD1Q',
    user_agent='Necessary_Ad_5588'
)
# VENTANA TEMPORAL (misma que Twitter)
fecha_plebiscito = datetime(2020, 10, 25)
fecha_inicio = datetime(2020, 10, 14)  # 11 días antes
fecha_fin = datetime(2020, 11, 26)      # 32 días después

print(f"Período de análisis: {fecha_inicio.date()} a {fecha_fin.date()}")
print(f"Total días: {(fecha_fin - fecha_inicio).days}")

# Recolectar posts
subreddits_chile = ['chile', 'RepublicadeChile']
keywords = [
    'plebiscito', 'apruebo', 'rechazo', 'constitución', 
    'nueva constitución', 'plebiscito 2020'
]
posts_data = []

for subreddit in subreddits_chile:
    print(f"\n📂 Recolectando de r/{subreddit}...")
    
    for keyword in keywords:
        print(f"   Buscando: '{keyword}'", end='')
        
        submissions = get_pushshift_data(subreddit, keyword, fecha_inicio, fecha_fin, size=500)
        
        for post in submissions:
            posts_data.append({
                'post_id': post.get('id'),
                'subreddit': subreddit,
                'titulo': post.get('title', ''),
                'texto': post.get('selftext', ''),
                'autor': post.get('author', '[deleted]'),
                'fecha': datetime.fromtimestamp(post.get('created_utc', 0)),
                'upvotes': post.get('score', 0),
                'num_comentarios': post.get('num_comments', 0),
                'url': post.get('full_link', ''),
                'keyword_encontrado': keyword
            })
        
        print(f" → {len(submissions)} posts")
        time.sleep(1)  # Respetar rate limits

print(f"\nTotal posts recolectados: {len(posts_data)}")

if len(posts_data) > 0:
    df_reddit = pd.DataFrame(posts_data)
    print(f"Período: {df_reddit['fecha'].min()} a {df_reddit['fecha'].max()}")
else:
    print(" No se encontraron datos")


Período de análisis: 2020-10-14 a 2020-11-26
Total días: 43

📂 Recolectando de r/chile...
   Buscando: 'plebiscito' → 75 posts
   Buscando: 'apruebo' → 76 posts
   Buscando: 'rechazo' → 63 posts
   Buscando: 'constitución' → 60 posts
   Buscando: 'nueva constitución' → 26 posts
   Buscando: 'plebiscito 2020' → 9 posts

📂 Recolectando de r/RepublicadeChile...
   Buscando: 'plebiscito' → 15 posts
   Buscando: 'apruebo' → 21 posts
   Buscando: 'rechazo' → 19 posts
   Buscando: 'constitución' → 18 posts
   Buscando: 'nueva constitución' → 6 posts
   Buscando: 'plebiscito 2020' → 1 posts

✅ Total posts recolectados: 389
Período: 2020-10-14 00:51:52 a 2020-11-23 23:01:56


In [16]:

print(f"\n Total posts recolectados: {len(posts_data)}")
df_reddit = pd.DataFrame(posts_data)
    
# Eliminar duplicados
df_reddit = df_reddit.drop_duplicates(subset=['post_id'])
    
print(f"\nPosts únicos: {len(df_reddit)}")
print(f"Período: {df_reddit['fecha'].min()} a {df_reddit['fecha'].max()}")

# ====================================
# CREAR Y PROCESAR DATAFRAME
# ====================================

# Crear DataFrame con los datos recolectados
df_reddit = pd.DataFrame(posts_data)

# Eliminar duplicados (algunos posts pueden aparecer en múltiples búsquedas)
print(f"\nPosts antes de eliminar duplicados: {len(df_reddit)}")
df_reddit = df_reddit.drop_duplicates(subset=['post_id'])
print(f"Posts únicos después de eliminar duplicados: {len(df_reddit)}")

# ====================================
# AGREGAR VARIABLES TEMPORALES
# ====================================

print("\n Agregando variables temporales...")

# Asegurar que 'fecha' sea datetime
df_reddit['fecha'] = pd.to_datetime(df_reddit['fecha'])

# Extraer componentes de fecha
df_reddit['fecha_solo'] = df_reddit['fecha'].dt.date
df_reddit['hora'] = df_reddit['fecha'].dt.hour
df_reddit['dia_semana'] = df_reddit['fecha'].dt.dayofweek  # ← CORREGIDO: usar dayofweek
df_reddit['nombre_dia'] = df_reddit['fecha'].dt.day_name()
df_reddit['mes'] = df_reddit['fecha'].dt.month
df_reddit['dia'] = df_reddit['fecha'].dt.day

# Calcular días respecto al plebiscito
fecha_plebiscito = pd.Timestamp('2020-10-25')
df_reddit['dias_desde_plebiscito'] = (df_reddit['fecha'] - fecha_plebiscito).dt.days

# Clasificar en ventanas temporales
def clasificar_ventana(dias):
    if dias < -30:
        return '1. Más de 30 días antes'
    elif -30 <= dias < -7:
        return '2. Entre 8-30 días antes'
    elif -7 <= dias < 0:
        return '3. Última semana antes'
    elif dias == 0:
        return '4. Día del plebiscito'
    elif 0 < dias <= 7:
        return '5. Primera semana después'
    else:
        return '6. Más de 7 días después'

df_reddit['ventana_temporal'] = df_reddit['dias_desde_plebiscito'].apply(clasificar_ventana)

# Agregar longitud del texto
df_reddit['longitud_titulo'] = df_reddit['titulo'].str.len()
df_reddit['longitud_texto'] = df_reddit['texto'].str.len()
df_reddit['tiene_texto'] = df_reddit['longitud_texto'] > 0

# ====================================
# INFORMACIÓN DEL DATAFRAME
# ====================================

print("\n" + "="*70)
print("INFORMACIÓN DEL DATAFRAME")
print("="*70)

print(f"\nDimensiones: {df_reddit.shape[0]} filas × {df_reddit.shape[1]} columnas")
print(f"Período: {df_reddit['fecha'].min()} a {df_reddit['fecha'].max()}")
print(f"Días de cobertura: {(df_reddit['fecha'].max() - df_reddit['fecha'].min()).days}")

print("\nColumnas del DataFrame:")
for i, col in enumerate(df_reddit.columns, 1):
    print(f"  {i:2d}. {col}")

print("\nPrimeras 5 filas:")
print(df_reddit.head())

print("\nÚltimas 5 filas:")
print(df_reddit.tail())



 Total posts recolectados: 389

Posts únicos: 289
Período: 2020-10-14 00:51:52 a 2020-11-23 23:01:56

Posts antes de eliminar duplicados: 389
Posts únicos después de eliminar duplicados: 289

 Agregando variables temporales...

INFORMACIÓN DEL DATAFRAME

Dimensiones: 289 filas × 21 columnas
Período: 2020-10-14 00:51:52 a 2020-11-23 23:01:56
Días de cobertura: 40

Columnas del DataFrame:
   1. post_id
   2. subreddit
   3. titulo
   4. texto
   5. autor
   6. fecha
   7. upvotes
   8. num_comentarios
   9. url
  10. keyword_encontrado
  11. fecha_solo
  12. hora
  13. dia_semana
  14. nombre_dia
  15. mes
  16. dia
  17. dias_desde_plebiscito
  18. ventana_temporal
  19. longitud_titulo
  20. longitud_texto
  21. tiene_texto

Primeras 5 filas:
  post_id subreddit                                             titulo  \
0  jqh61j     chile  Dos Venezolanos predicen el futuro de Chile tr...   
1  jpjh09     chile  Sobre el tema de los manifestantes radicales, ...   
2  jo52pp     chile     

In [18]:
archivo_csv = '../data/processed/reddit_posts_plebiscito_2020.csv'
df_reddit.to_csv(archivo_csv, index=False, encoding='utf-8')
print(f"\n CSV guardado: {archivo_csv}")
print(f"   Tamaño: {len(df_reddit)} filas × {len(df_reddit.columns)} columnas")


 CSV guardado: ../data/processed/reddit_posts_plebiscito_2020.csv
   Tamaño: 289 filas × 21 columnas
